In [ ]:
install.packages(c("openxlsx", "dplyr", "tidyr", "ggpubr", "RColorBrewer", "cowplot", "stringr", "jsonlite", "grid"))


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘grid’ is a base package, and should not be updated”
also installing the dependencies ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘corrplot’, ‘car’, ‘Rcpp’, ‘ggrepel’, ‘ggsci’, ‘ggsignif’, ‘gridExtra’, ‘polynom’, ‘rstatix’




In [ ]:
library(dplyr)
library(tidyr)
library(openxlsx)
# library(ggplot2)
# library(plotly)
library(ggpubr)
library(RColorBrewer)
# library(rcartocolor)
library(cowplot)
library(stringr)
# library(webr)
#read muscle information table
# library(gt)
library(jsonlite)
library("grid")
library(readxl)

In [ ]:
# Utility Functions
saveFig <- function(fig,name,wdth,hght){
  # Save final image
  savename <- paste(name,".png",sep = '')
  ggsave(filename = savename,
       plot = fig,
       width = wdth,
       height = hght,
       units = "in",
       dpi = 300)

}


corrections <- function(df){
  #this functions removes studies and rows

  # Remove rows based on IDs
df <- subset(df, !(CovidenceID == 4184))
df <- subset(df, !(CovidenceID == 2679))
  # Remove rows where ID is 2837 and CSA is 'ACSA'
df <- subset(df, !(CovidenceID == 2837 & CSA == 'ACSA'))
df <- subset(df, !(CovidenceID == 2995 & CSA == 'ACSA'))

return (df)
}

#<---------------load and process paperstats - paperstats_r, paperstats_r_filtered

loadPaperStats <- function(){
  paperstats <- read.csv("Paperstats.csv")
  #remove mistakes
  paperstats=corrections(paperstats)

  # Select specific columns and rename them
  paperstats_r <- paperstats %>%
  select(CovidenceID = CovidenceID, aveAge = "aveAge..years.",aveHeight = "aveHeight..cm.",
         aveWeight="aveWeight..kg.",N="X.Number.of.subjects..N",Muscle=Muscle,Type=Type,
         Gender=Gender,CSA=CSA,Miscellaneous=Miscellaneous,volORstim="Voluntary..V..or.Stimulated..S..Force.measurement",
         combineTitle="Title_x", studyID="Study.ID", country="Country", studyDesign="Study.design", year="Year",
         journal="Journal",aveST="aveST..N.cm2.",musclename="name")

  # colnames(paperstats)

  paperstats_r$journal <- as.factor(paperstats_r$journal)
  paperstats_r$musclename <- as.factor(paperstats_r$musclename)


  paperstats_r <- paperstats_r %>% drop_na(aveST) # remove muscles that had no ST vals
  paperstats_r <- paperstats_r[!paperstats_r$Muscle == '', ] # remove empty muscle rows
  # levels(paperstats_r$journal)[levels(paperstats_r$journal) == "J Appl Physiol (1985)"] <- "J Appl Physiol" # rename this journal
  levels(paperstats_r$journal)[levels(paperstats_r$journal) == "GNB"] <- "GNB2012 Conference" # rename this journal
  levels(paperstats_r$journal)[levels(paperstats_r$journal) == "Journal of Applied Physiology"] <- "J Appl Physiol" # rename this journal
  levels(paperstats_r$journal)[levels(paperstats_r$journal) == "The journal of physiology"] <- "J Physiol" # rename this journal

  levels(paperstats_r$country)[levels(paperstats_r$country) == "UK"] <- "United Kingdom"


  #remove repeated to get proper paperstats
  paperstats_r_filtered <- paperstats_r %>% distinct(`CovidenceID`, `Muscle`, .keep_all = TRUE)

  # colnames(paperstats_r)

  output<-list(paperstats_r,paperstats_r_filtered)
  return(output)

}

# convert to just using one doc
# test <- read_excel("./included/ST_review_ALLdata_Lomas_processed.csv", sheet = "Plotdata")


#<---------------nowload and process plotdata - data_r

loadPlotdata <- function(){
  data <- read.csv("Plotdata.csv")

  #remove mistakes
  data=corrections(data)

  # Select specific columns and rename them
  data_r <- data %>%
  select(CovidenceID = CovidenceID, aveAge = "aveAge..years.",aveHeight = "aveHeight..cm.",
         aveWeight="aveWeight..kg.",N="X.Number.of.subjects..N",Muscle=Muscle,Type=Type,
         Gender=Gender,CSA=CSA,Miscellaneous=Miscellaneous,aveVol="aveVol..cm3.",
         aveFL="aveFL..cm.",avePCSA="avePCSA..cm2.",aveTorque="aveTorque..Nm.",
         aveForce="aveForce..N.",aveST="aveST..N.cm2.", agetype="Age.type", trainingtype="Training.type", muscleName="name",
         jointType="Joint.type", grav="antigrav.grav", jointAction="Joint.action", anatomicalLocation="anatomical.location", FastSlow="fast.slow", Score="Score",mv_method="Muscle.volume.method",fl_method="Fascicle.length.method",pa_method="Pennation.angle.method",t_method="Torque.method", ma_method="Moment.arm.method", pcsa_method="PCSA.method", f_method="Force.method")

  data_r <- data_r %>% drop_na(aveST) # remove muscles that had no ST vals
  # Define the order you want for the legend
  desired_order <- c("CHD", "YNG", "MID", "OLD")


  # Convert to factors
  data_r$Gender <- as.factor(data_r$Gender)
  data_r$Muscle <- as.factor(data_r$Muscle)
  data_r$agetype <- factor(data_r$agetype, levels = desired_order)
  data_r$trainingtype <- as.factor(data_r$trainingtype)
  data_r$muscleName <- as.factor(data_r$muscleName)
  data_r$jointType <- as.factor(data_r$jointType)
  data_r$jointAction <- as.factor(data_r$jointAction)
  data_r$anatomicalLocation <- as.factor(data_r$anatomicalLocation)

  # # Returns string without leading white space
  # trim.leading <- function (x)  sub("^\\s+", "", x)
  #
  # # Returns string without trailing white space
  # trim.trailing <- function (x) sub("\\s+$", "", x)
  #
  # # Returns string without leading or trailing white space
  # trim <- function (x) gsub("^\\s+|\\s+$", "", x)
  #
  # data_r$Gender <- trim(data_r$Gender)


  levels(data_r$agetype)[levels(data_r$agetype) == "CHD"] <- "Child (<18)"
  levels(data_r$agetype)[levels(data_r$agetype) == "YNG"] <- "Young (18-34)"
  levels(data_r$agetype)[levels(data_r$agetype) == "MID"] <- "Middle (35-65)"
  levels(data_r$agetype)[levels(data_r$agetype) == "OLD"] <- "Old (>65)"

  data_r$trainingtype2 <- data_r$trainingtype

  levels(data_r$trainingtype2)[levels(data_r$trainingtype2) == "Untrained"] <- "Normal"

  levels(data_r$Gender)[levels(data_r$Gender) == "M"] <- "Male"
  levels(data_r$Gender)[levels(data_r$Gender) == "F"] <- "Female"
  levels(data_r$Gender)[levels(data_r$Gender) == "B"] <- "Male"
  levels(data_r$Gender)[levels(data_r$Gender) == "G"] <- "Female"


  levels(data_r$anatomicalLocation)[levels(data_r$anatomicalLocation) == "Thigh"] <- "Upper leg"


  levels(data_r$jointAction)[levels(data_r$jointAction) == "knee extensor"] <- "Knee Ext."
  levels(data_r$jointAction)[levels(data_r$jointAction) == "elbow extensor"] <- "Elbow Ext."
  levels(data_r$jointAction)[levels(data_r$jointAction) == "ankle plantar flexor"] <- "Plantarflex"
  levels(data_r$jointAction)[levels(data_r$jointAction) == "elbow flexor"] <- "Elbow Flex."
  levels(data_r$jointAction)[levels(data_r$jointAction) == "ankle dorsiflexor"] <- "Dorsiflex"
  levels(data_r$jointAction)[levels(data_r$jointAction) == "hip adductor"] <- "Hip Adduct."

  data_r$mv_method[data_r$mv_method == ""] <- "MV_none"
  data_r$fl_method[data_r$fl_method == ""] <- "fl_none"
  data_r$pa_method[data_r$pa_method == ""] <- "Pa_none"
  data_r$t_method[data_r$t_method == ""] <- "Torq_none"
  data_r$ma_method[data_r$ma_method == ""] <- "MA_none"
  data_r$pcsa_method[data_r$pcsa_method == ""] <- "PCSA_none"
  data_r$f_method[data_r$f_method == ""] <- "Force_none"


  #remove repeated to get proper paperstats
  data_r_filtered <- data_r %>% distinct(`CovidenceID`, `Muscle`, .keep_all = TRUE)

  # colnames(paperstats_r)

  output<-list(data_r,data_r_filtered)

  return(output)

}



result <- loadPaperStats()
# Access the returned variables from the list
paperstats_r <- result[[1]]
# paperstats_r_filtered <-result[[2]]



result2 <- loadPlotdata()
# Access the returned variables from the list
data_r <- result2[[1]]
# data_r_filtered <-result2[[2]]



# now join both data sets
# colnames(data_r)

#<-----------strict studystats vs muscle analysis
# NB muscle rows are distinct. so can only aggregate variables related to distinct muscles. so age range cannot be coinsidered since those rows are removed

# Join the data frames based on the 'ID' column
# result_df <- paperstats_r_filtered %>%
#   left_join(data_r_filtered, by = c("CovidenceID","Muscle")) %>%
#   select(muscleName, jointType,jointAction,anatomicalLocation)


result_df <- data_r %>%
  select(muscleName, jointType,jointAction,anatomicalLocation)

paperstats_r <- cbind(paperstats_r, result_df)

# unique(data_r$jointAction)

# unique(paperstats_r$journal)



# data <- read.csv("./included/Plotdata.csv")
# colnames(data)
#
#
#  mv_method="Muscle.volume.method",fl_method="Fascicle.length.method",pa_method-"Pennation.angle.method"
#  t_method="Torque.method", ma_method="Moment.arm.method", pcsa_method="PCSA.method", f_method="Force.method"
muscle_order <- c("Biceps Brachii", "Brachialis", "Brachioradialis", "Elbow Flexors",
                  "Triceps Brachii", "Elbow Extendors",
                  "Quadriceps Femoris", "Rectus Femoris","Vastus Intermedius", "Vastus Medialis", "Vastus Lateralis",
                  "Gracilis",
                  "Plantar Flexors","Soleus","Gastrocnemius Lateralis", "Gastrocnemius Medialis","Triceps Surae","Dorsiflexor", "Tibialis Anterior")

custom_colors<-c('black','forestgreen', 'red2', 'orange', 'cornflowerblue',
                'magenta', 'darkolivegreen4', 'indianred1', 'tan4', 'darkblue',
                'mediumorchid1','firebrick4',  'yellowgreen', 'lightsalmon', 'tan3',
                "tan1",'darkgray', 'wheat4', '#DDAD4B', 'chartreuse',
                'seagreen1', 'moccasin', 'mediumvioletred', 'seagreen','cadetblue1',
                "darkolivegreen1" ,"tan2" ,   "tomato3" , "#7CE3D8","gainsboro")

custom_colors <- c(brewer.pal(6, "Set2"),custom_colors)


In [ ]:
muscle_order2 <- c("BIC", "BRA", "BRD", "EF",
                  "TRI", "EE",
                  "QF", "RF","VI", "VM", "VL",
                  "GRA",
                  "PF","SOL","GL", "GM","TS","DF", "TA")

paperstats_r <- paperstats_r %>%
  mutate(Muscle = factor(Muscle, levels = muscle_order2))

## add stats to boxplots

In [ ]:
plot_box_no_grps <- function(xcol,xlabel){
   my_comparisons <- list( c("Male", "Female") )


  grp2 <- ggboxplot(data_r, x = xcol, y = "aveST",
                palette ="jco",add = "jitter",
                xlab = xlabel)+ stat_compare_means(comparisons = xcol)


 grp2 <- grp2+theme_pubr() + labs(y = expression("Specific tension "~(N/cm^2)))+theme(axis.text = element_text(size = 12))



   # stat_compare_means(comparisons = my_comparisons)+ # Add pairwise comparisons p-value
  # stat_compare_means(label.y = 50)

 return (grp2)

}

data_r <- data_r %>%
  mutate(anatomicalLocation = factor(anatomicalLocation, levels = c("Upper arm","Upper leg", "Lower leg")))

data_r <- data_r %>%
  mutate(jointAction = factor(jointAction, levels = c("Elbow Flex.","Elbow Ext.", "Hip Adduct.", "Knee Ext.", "Plantarflex", "Dorsiflex")))

box1c <- plot_box_no_grps("anatomicalLocation","Anatomical location")+
  theme(legend.position = "none")
box2c <- plot_box_no_grps("jointAction","Function")
box3c <- plot_box_no_grps("trainingtype","Training status")+
  theme(legend.position = "none")
box4c <- plot_box_no_grps("jointType","Mono/Biarticular")+
  theme(legend.position = "none")
box5c <- plot_box_no_grps("FastSlow","Fiber type (Fast/Slow)")+
  theme(legend.position = "none")





In [ ]:
ggboxplot(data_r, x = "anatomicalLocation", y = "aveST",
          palette = "jco", add = "jitter",
          xlab = "Anatomical location") +
  stat_compare_means(comparisons = list(c("Upper leg", "Lower leg"), c("Upper arm", "Lower leg")),
                     ref.group = ".all.",method = "t.test", label = "p.", map_signif_level=TRUE) +
  theme_pubr()

                  # stat_compare_means(method = "t.test", label = "p.format",map_signif_level=TRUE)


    # stat_compare_means(ref.group = ".all.",method = "t.test", label = "p.format", map_signif_level=TRUE)


## review scoring

In [ ]:
scored_Data <- read.csv("Plotdata_scored.csv")


In [ ]:
column_name_str <- "
         aveAge = 'aveAge..years.',
         aveHeight = 'aveHeight..cm.',
         aveWeight = 'aveWeight..kg.',
         N = 'X.Number.of.subjects..N',
         aveVol = 'aveVol..cm3.',
         aveFL = 'aveFL..cm.',
         avePCSA = 'avePCSA..cm2.',
         aveTorque = 'aveTorque..Nm.',
         aveForce = 'aveForce..N.',
         aveST = 'aveST..N.cm2.',
         agetype = 'Age.type',
         trainingtype = 'Training.type',
         muscleName = 'name',
         jointType = 'Joint.type',
         grav = 'antigrav.grav',
         jointAction = 'Joint.action',
         anatomicalLocation = 'anatomical.location',
         FastSlow = 'fast.slow',
         Score = 'Score',
         mv_method = 'Muscle.volume.method',
         fl_method = 'Fascicle.length.method',
         pa_method = 'Pennation.angle.method',
         t_method = 'Torque.method',
         ma_method = 'Moment.arm.method',
         pcsa_method = 'PCSA.method',
         f_method = 'Force.method'"

# Convert the string to a named character vector
column_names <- setNames(strsplit(gsub("'", "", column_name_str), ",\\s*")[[1]], NULL)

# Change column names using a loop
for (old_name in names(column_names)) {
  colnames(scored_Data)[colnames(scored_Data) == old_name] <- column_names[old_name]
}

In [ ]:
scored_Data <- scored_Data %>% distinct(`CovidenceID`, `newScore`, .keep_all = TRUE)

In [ ]:
fig10complete <- ggplot(scored_Data, aes(x = 0, y = aveST)) +
  geom_boxplot(outlier.shape = NA, width = 0.08) +
  geom_jitter(aes(x = 0, y = aveST), width = 0.009, alpha = 0.5) +
  stat_summary(aes(x = 0), fun = median, geom = "point", shape = 18, size = 3, color = "red") +
  geom_text(data = scored_Data %>% group_by(newScore) %>% summarize(med_aveST = median(aveST)),
            aes(x = 0, y = med_aveST, label = round(med_aveST, 2)),
            vjust = -0.5, hjust = -0.2, size = 4, color = "red") +
   ggtitle("Assumption Count") +
  facet_wrap(~Score, scales = "fixed", nrow = 1) + #labeller = as_labeller(facet_names)
  theme_minimal() +
  labs(y = expression("Specific tension "~(N/cm^2))) +
  theme(axis.title.x = element_blank(), axis.text.x = element_blank(), axis.ticks.x = element_blank())


 fig10complete
#  saveFig(fig10complete,'assumption_count',6,9)

In [ ]:

colnames(scored_Data)

[1] "CovidenceID"                     "aveAge..years."                 
 [3] "aveHeight..cm."                  "aveWeight..kg."                 
 [5] "X.Number.of.subjects..N"         "Muscle"                         
 [7] "Type"                            "Gender"                         
 [9] "Voluntary..V..or.Stimulated..S." "aveVol..cm3."                   
[11] "aveFL..cm."                      "avePCSA..cm2."                  
[13] "aveTorque..Nm."                  "aveForce..N."                   
[15] "aveST..N.cm2."                   "Muscle.volume.method"           
[17] "Fascicle.length.method"          "Pennation.angle.method"         
[19] "CSA"                             "PCSA.method"                    
[21] "Torque.method"                   "Moment.arm.method"              
[23] "Force.method"                    "Specific.Tension.method"        
[25] "Overall.note"                    "Miscellaneous"                  
[27] "Score"                           "Age.type"                       
[29] "Training.type"                   "Muscle.1"                       
[31] "name"                            "fast.slow"                      
[33] "Joint.type"                      "antigrav.grav"                  
[35] "Joint.action"                    "secondary.joint.action"         
[37] "anatomical.location"             "Torque.Force.method"            
[39] "Muscle.volume.method_score"      "Torque.Force.method_score"      
[41] "Fascicle.length.method_score"    "Pennation.angle.method_score"   
[43] "PCSA.method_score"               "Moment.arm.method_score"        
[45] "Specific.Tension.method_score"   "newScore"                       
[47] "Covidence.."                     "Study.ID"                       
[49] "Title_y"                         "Country"                        
[51] "Study.design"                    "Authors"                        
[53] "Year"                            "Journal"                        
[55] "aveAge"                          "aveHeight"                      
[57] "aveWeight"                       "N"                              
[59] "aveVol"                          "aveFL"                          
[61] "avePCSA"                         "aveTorque"                      
[63] "aveForce"                        "aveST"                          
[65] "agetype"                         "trainingtype"                   
[67] "muscleName"                      "jointType"                      
[69] "grav"                            "jointAction"                    
[71] "anatomicalLocation"              "FastSlow"                       
[73] "mv_method"                       "fl_method"                      
[75] "pa_method"                       "t_method"                       
[77] "ma_method"                       "pcsa_method"                    
[79] "f_method"

In [ ]:
scored_Data

# create Methods df

In [ ]:
scored_Data <- read.csv("Plotdata_scored.csv")
llm_data<-read_excel("/content/zephyr-7b-alpha-Q5_K_M.xlsx")
column_name_str <- "
         aveAge = 'aveAge..years.',
         aveHeight = 'aveHeight..cm.',
         aveWeight = 'aveWeight..kg.',
         N = 'X.Number.of.subjects..N',
         aveVol = 'aveVol..cm3.',
         aveFL = 'aveFL..cm.',
         avePCSA = 'avePCSA..cm2.',
         aveTorque = 'aveTorque..Nm.',
         aveForce = 'aveForce..N.',
         aveST = 'aveST..N.cm2.',
         agetype = 'Age.type',
         trainingtype = 'Training.type',
         muscleName = 'name',
         jointType = 'Joint.type',
         grav = 'antigrav.grav',
         jointAction = 'Joint.action',
         anatomicalLocation = 'anatomical.location',
         FastSlow = 'fast.slow',
         Score = 'Score',
         mv_method = 'Muscle.volume.method',
         fl_method = 'Fascicle.length.method',
         pa_method = 'Pennation.angle.method',
         t_method = 'Torque.method',
         ma_method = 'Moment.arm.method',
         pcsa_method = 'PCSA.method',
         f_method = 'Force.method'"

# Convert the string to a named character vector
column_names <- setNames(strsplit(gsub("'", "", column_name_str), ",\\s*")[[1]], NULL)

# Change column names using a loop
for (old_name in names(column_names)) {
  colnames(scored_Data)[colnames(scored_Data) == old_name] <- column_names[old_name]
}

In [ ]:
scored_Data <- scored_Data %>% distinct(`CovidenceID`, `newScore`, .keep_all = TRUE)

ERROR: Error in scored_Data %>% distinct(CovidenceID, newScore, .keep_all = TRUE): could not find function "%>%"
